## 💻 Unity Catalog quick demo to Blue Harvest data domain
### Repository with step by step to create a demo for [Databricks Unity Catalog](https://learn.microsoft.com/en-us/azure/databricks/data-governance/unity-catalog/)



### 📝   Details:

Dataset is online retail platform that captures customer information, customer orders and customer events. We will create a catalog, then create tables and learn how Unity Catalog allows you to securely discover, access and collaborate on trusted data and AI assets

### 📌 Ingestion and transformation

#### list databricks datasets demo - on retail org

In [ ]:
%fs ls /databricks-datasets/retail-org/


path,name,size,modificationTime
dbfs:/databricks-datasets/retail-org/README.md,README.md,1678,1602776953000
dbfs:/databricks-datasets/retail-org/active_promotions/,active_promotions/,0,1699457724942
dbfs:/databricks-datasets/retail-org/company_employees/,company_employees/,0,1699457724942
dbfs:/databricks-datasets/retail-org/customers/,customers/,0,1699457724942
dbfs:/databricks-datasets/retail-org/loyalty_segments/,loyalty_segments/,0,1699457724942
dbfs:/databricks-datasets/retail-org/products/,products/,0,1699457724942
dbfs:/databricks-datasets/retail-org/promotions/,promotions/,0,1699457724942
dbfs:/databricks-datasets/retail-org/purchase_orders/,purchase_orders/,0,1699457724942
dbfs:/databricks-datasets/retail-org/sales_orders/,sales_orders/,0,1699457724942
dbfs:/databricks-datasets/retail-org/sales_stream/,sales_stream/,0,1699457724942


#### get user id

In [ ]:
user_id = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
user_id = ''.join(filter(str.isdigit, user_id))
print(user_id)

12


#### print read me to understand better the retail org dataset

In [ ]:
f = open('/dbfs/databricks-datasets/retail-org/README.md', 'r')
print(f.read())

Synthetic Retail Dataset
This dataset is a collection of files representing different dimensions and facts for a retail organization.

Provenance
This dataset was generated by Databricks.

Data Set Information
* Sales Orders: **sales_orders/sales_orders.json** records the customers' originating purchase order.
* Purchase Orders: **purchase_orders/purchase_orders.xml** contains the raw materials that are being purchased.
* Products: **products/products.csv** contains products that the company sells.
* Goods Receipt: **goods_receipt/goods_receipt.parquet** contains the arrival time of purchased orders.
* Customers: **customers/customers.csv** contains those customers who are located in the US and are buying the finished products.
* Suppliers: **suppliers/suppliers.csv** contains suppliers that provide raw materials in the US.
* Sales Stream: **sales_stream/sales_stream.json/** is a folder containing JSON files for streaming purposes.
* Promotions: **promotions/promotions.csv** contains a

#### Create and ingest Active Promotions at bronze layer

In [ ]:
active_promo = spark.read.parquet('/databricks-datasets/retail-org/active_promotions/')

active_promo.write.format("delta").mode("overwrite").saveAsTable(f"unity_catalog_blue_harvest.default.active_promotions_bronze")

#### Create and ingest Customers at bronze layer

In [ ]:
customers = spark.read.csv('dbfs:/databricks-datasets/retail-org/customers/', header = True)

customers.write.format("delta").mode("overwrite").saveAsTable(f"unity_catalog_blue_harvest.default.customers_bronze")

#### Create and ingest Suppliers at bronze layer

In [ ]:
suppliers = spark.read.csv('dbfs:/databricks-datasets/retail-org/suppliers/', header = True)

suppliers.write.format("delta").mode("overwrite").saveAsTable(f"unity_catalog_blue_harvest.default.suppliers_bronze")

#### Create and ingest Loyalty segment at bronze layer

In [ ]:
loyalty_segment = spark.read.csv('dbfs:/databricks-datasets/retail-org/loyalty_segments/', header = True)

loyalty_segment.write.format("delta").mode("overwrite").saveAsTable(f"unity_catalog_blue_harvest.default.loyalty_segment_bronze")

#### Create and ingest Sales Orders at bronze layer

In [ ]:
sales_orders = spark.read.json('/databricks-datasets/retail-org/sales_orders/')

sales_orders.write.format("delta").mode("overwrite").saveAsTable(f"unity_catalog_blue_harvest.default.sales_orders_bronze")

#### Create and ingest Retail at bronze layer

In [ ]:
products = spark.read.option("delimiter", ";").csv('/databricks-datasets/retail-org/products/', header = True)

products.write.format("delta").mode("overwrite").saveAsTable(f"unity_catalog_blue_harvest.default.products_bronze")

In [ ]:
%sql
CREATE OR REPLACE TABLE unity_catalog_blue_harvest.default.products_silver AS
SELECT product_category, SUM(sales_price) total_sales
FROM unity_catalog_blue_harvest.default.products_bronze
GROUP BY product_category;

SELECT * FROM unity_catalog_blue_harvest.default.products_silver

product_category,total_sales
Rony,7087.99993223018
Olitscreens,2420.313611111111
Sioneer,1785.2410971394
Ankyo,1386.7884807692305
Mannheiser,3274.2211666666662
Opple,9069.704107241152
Mowepro,215.34988906926404
Elpine,1295.779596908939
Karsair,927.05225
Reagate,280.6986834733893


In [ ]:
%sql
CREATE OR REPLACE TABLE unity_catalog_blue_harvest.default.top10_customer_gold AS

SELECT  t2.customer_name, count(t1.number_of_line_items) as sum_of_items
FROM unity_catalog_blue_harvest.default.sales_orders_bronze t1
inner join unity_catalog_blue_harvest.default.customers_bronze t2
on
t1.customer_id = t2.customer_id
GROUP BY 1
order by 2 desc
limit 10;

SELECT * FROM unity_catalog_blue_harvest.default.top10_customer_gold


customer_name,sum_of_items
digital lifestyle solutions,69
statusdigital,65
"atlanta digital studio, llc",65
genesis electronics recycling,64
rpm optoelectronics,63
digital photo solutions ltd,62
pro digital solutions,60
digital attic,60
"bradsworth digital solutions, inc",59
epi-electrochemical products inc,58


### 📌   Now start grant permission and the whole magic of unity catalog


In [ ]:
%sql
SHOW CATALOGS;

catalog
hive_metastore
main
samples
system
unity_catalog_blue_harvest


In [ ]:
grant use schema, select on database x to `activedirectory group value`
grant use schema, select on database x to `user`


### here I can segregate the access of each business unit per users

In [ ]:
%sql

CREATE OR REPLASE VIEW DATABASE.SCHEMA.viewname AS

SELECT
  * FROM DATABASE.SCHEMA.TABLE
WHERE
  CASE
  --- SPECIFIC USER CAN READ all
  WHEN CURRENT_USER() = 'USER' THEN True
  --- REST CAN SEE NOTHING
  ELSE FALSE
END

### here I can segregate the access of each business unit per active directory group, also this active directory group do not need to be maintained by databricks

In [ ]:
%sql

CREATE OR REPLASE VIEW DATABASE.SCHEMA.viewname AS
SELECT
* FROM DATABASE.SCHEMA.TABLE
WHERE
  CASE
      WHEN IS_MEMBER('ACTIVE_DIRECTORY_GROUP_FOR_SPAIN_USERS') = 1 THEN INTRA_CO_BU_CD like '%ES%'
      WHEN IS_MEMBER('ACTIVE_DIRECTORY_GROUP_FOR_JAPAN_USERS') = 1 THEN INTRA_CO_BU_CD like '%JP%'
      WHEN IS_MEMBER('ACTIVE_DIRECTORY_GROUP_FOR_ROUMAINE_USERS') = 1 THEN INTRA_CO_BU_CD like '%RO%'
      WHEN IS_MEMBER('ACTIVE_DIRECTORY_GROUP_FOR_POLAND_USERS') = 1 THEN INTRA_CO_BU_CD like '%PL%'
      WHEN IS_MEMBER('ACTIVE_DIRECTORY_GROUP_FOR_CZECHIA_USERS') = 1 THEN INTRA_CO_BU_CD like '%CZ%'
      WHEN IS_MEMBER('ACTIVE_DIRECTORY_GROUP_FOR_TURKEY_USERS') = 1 THEN INTRA_CO_BU_CD like '%TR%'
      WHEN IS_MEMBER('ACTIVE_DIRECTORY_GROUP_FOR_GERMANY_USERS') = 1 THEN INTRA_CO_BU_CD like '%GR%'
      ELSE FALSE
  END


## Data Lineage

### Lineage
![Lineage](https://github.com/QuentinAmbard/databricks-demo/blob/main/product_demos/uc/lineage/uc-lineage-slide.png?raw=true "Lineage")

###Column Lineage
![Lineage](https://github.com/QuentinAmbard/databricks-demo/blob/main/product_demos/uc/lineage/lineage-column.gif?raw=true "Lineage")

#### Table Lineage
![Lineage](https://github.com/QuentinAmbard/databricks-demo/blob/main/product_demos/uc/lineage/lineage-table.gif?raw=true "Lineage")

## Details

### 🛠  Research and Demo
Table ACL & Row and Column Level Security With Unity Catalog
https://notebooks.databricks.com/demos/uc-01-acl/index.html

Access Data on External Locations
https://notebooks.databricks.com/demos/uc-02-external-location/index.html

Data Lineage With Unity Catalog
https://notebooks.databricks.com/demos/uc-03-data-lineage/00-UC-lineage.html

System Tables: Billing Forecast, Usage Analytics, and Access Auditing With Databricks Unity Catalog
https://notebooks.databricks.com/demos/uc-04-system-tables/index.html

Upgrade Table to Unity Catalog
https://notebooks.databricks.com/demos/uc-05-upgrade/index.html

Azure Databricks Unity catalog
https://learn.microsoft.com/en-us/azure/databricks/data-governance/unity-catalog/data-lineage

Lineage Demo
https://app.getreprise.com/launch/MnqjQDX/

https://github.com/databricks-demos/dbdemos

Ebook Governance and Unity catalog
https://www.databricks.com/sites/default/files/2023-10/final_data-and-ai-governance.6sept2023.pdf

https://www.databricks.com/resources/demos/tutorials/lakehouse-platform/dbdemos-hls-patient-readmission-health?itm_data=demo_center
https://www.databricks.com/resources/demos/tutorials/governance/table-acl-and-dynamic-views-with-uc?itm_data=demo_center
https://www.databricks.com/resources/demos/tours/governance/query-federation-product-tour?itm_data=demo_center
https://www.databricks.com/resources/demos/videos/governance/unity-catalog-demo?itm_data=demo_center


How to use demo databricks datasets
https://www.databricks.com/resources/demos/library?itm_data=demo_center
Demo
https://app.getreprise.com/launch/Q6ojw2y/



Quick step
%pip install dbdemos

import dbdemos
dbdemos.install('uc-03-data-lineage')

-----

Going further with Data governance & security
By bringing all your data assets together, Unity Catalog let you build a complete and simple governance to help you scale your teams.

Unity Catalog can be leveraged from simple GRANT to building a complete datamesh organization.


Fine-grained ACL: row/column level access
Need more advanced control? You can chose to dynamically change your table output based on the user permissions: dbdemos.intall('uc-01-acl')

Secure external location (S3/ADLS/GCS)
Unity Catatalog let you secure your managed table but also your external locations: dbdemos.intall('uc-02-external-location')

Lineage
UC automatically captures table dependencies and let you track how your data is used, including at a row level: dbdemos.intall('uc-03-data-lineage')

This leat you analyze downstream impact, or monitor sensitive information across the entire organization (GDPR).

Audit log
UC captures all events. Need to know who is accessing which data? Query your audit log: dbdemos.intall('uc-04-audit-log')

This leat you analyze downstream impact, or monitor sensitive information across the entire organization (GDPR).

Upgrading to UC
Already using Databricks without UC? Upgrading your tables to benefit from Unity Catalog is simple: dbdemos.intall('uc-05-upgrade')

Sharing data with external organization
Sharing your data outside of your Databricks users is simple with Delta Sharing, and doesn't require your data consumers to use Databricks: dbdemos.intall('delta-sharing-airlines')
----
Research

https://www.databricks.com/resources/demos/tutorials/lakehouse-platform/dbdemos-hls-patient-readmission-health?itm_data=demo_center
https://www.databricks.com/resources/demos/tutorials/governance/table-acl-and-dynamic-views-with-uc?itm_data=demo_center
https://www.databricks.com/resources/demos/tours/governance/query-federation-product-tour?itm_data=demo_center
https://www.databricks.com/resources/demos/videos/governance/unity-catalog-demo?itm_data=demo_center

#  📚 Ebook
https://www.databricks.com/sites/default/files/2023-10/final_data-and-ai-governance.6sept2023.pdf

## 👨🏻‍🔧 Details
https://www.databricks.com/blog/2022/04/20/announcing-gated-public-preview-of-unity-catalog-on-aws-and-azure.html



### Audit

In [ ]:
## Links

https://medium.com/@24chynoweth/databricks-system-tables-an-introduction-e11a06872405
https://notebooks.databricks.com/demos/uc-01-acl/index.html
https://notebooks.databricks.com/demos/uc-04-system-tables/index.html#


👨🏽‍🏫 Created by [Diego Lopes](mailto:lopesdiego12@gmail.com)